# Comp3162 Lab 3
##Trishanna Ford - 620147893


###Section 1

In [37]:
!pip install mysql.connector

In [38]:
import mysql.connector
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [39]:
connection = mysql.connector.connect(
    host='database-comp3162.cqb8o27401uq.us-east-2.rds.amazonaws.com',
    user='comp3162user',
    password='Password876',
    port=3306,
    database='comp3162'
)

In [40]:
cursor = connection.cursor()
cursor.execute("Select * from Patients")
rows = cursor.fetchall()
rows = [row for row in rows if None not in row]
columns = [col[0] for col in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df.to_csv('task1-620147893.csv', index=False)
cursor.close()
connection.close()

###Section 2

In [41]:
url = "http://universities.hipolabs.com/search?country=India"
response = requests.get(url)

if response.status_code==200:
  json_data = response.json()
  rows = []
  for item in json_data:
    name = item.get("name")
    state_province = item.get("state_province")
    rows.append({"Name": name, "State/Province": state_province})
    df2 = pd.concat([pd.DataFrame([row]) for row in rows], ignore_index=True)
df2.to_csv('task2-620147893.csv', index=False)

###Section 3

In [43]:
all_data = []
def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('article', class_='product_pod')
    for container in articles:
        title = container.find('h3').text.strip()
        star_rating_mapping = {
            'One': 1,
            'Two': 2,
            'Three': 3,
            'Four': 4,
            'Five': 5
        }
        star_rating_class = container.find('p', class_='star-rating')
        if star_rating_class:
            star_rating_class = star_rating_class['class'][1]
            star_rating = star_rating_class.capitalize()
            star_rating_numeric = star_rating_mapping.get(star_rating)
        else:
            star_rating_numeric = None
        price = container.find('p', class_='price_color').text.strip()
        all_data.append((title, star_rating_numeric, price))
    return all_data


for page_num in range(1, 3):
    page_url = f"https://books.toscrape.com/catalogue/category/books/mystery_3/page-{page_num}.html"  # Update the URL structure accordingly
    data = scrape_page(page_url)
    all_data.extend(data)

df3 = pd.DataFrame(all_data,columns=['Title','Star Rating','Price'])
df3.to_csv('task3-620147893.csv', index=False)